In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers datasets evaluate torch nltk
!pip install transformers datasets evaluate torch nltk rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9b56d38011c0943d249555597443d90eb5a2aca7c3dae10a3da24326d882edb6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


## Original Dataset

In [ ]:
# Load and split the dataset
# dataset = load_dataset("toughdata/quora-question-answer-dataset")
# dataset = dataset["train"].train_test_split(test_size=0.2)

## Preprocessed Dataset

In [ ]:
import pandas as pd

# Load preprocessed datasets
df_test_preprocessed = pd.read_csv('/kaggle/input/quora-preprocessed/preprocessed_test_dataset.csv')
df_train_preprocessed = pd.read_csv('/kaggle/input/quora-preprocessed/preprocessed_train_dataset.csv')

from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(df_train_preprocessed)
test_dataset = Dataset.from_pandas(df_test_preprocessed)
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


print("Datasets converted back to original format for model training.")

In [ ]:
import nltk
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import evaluate

# Ensure GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load and split the dataset
# dataset = load_dataset("toughdata/quora-question-answer-dataset")
# dataset = dataset["train"].train_test_split(test_size=0.2)

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

def preprocess_function(examples):
    """Tokenize the text."""
    inputs = [q + " " + a for q, a in zip(examples["question"], examples["answer"])]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["question", "answer"])  # remove original columns
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")  # rename label column

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


In [ ]:
trainer.save_model('./results/final_model')

input_text = "Can longer ooutputs be generated?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))